# 4.3 質問応答のInstruction-Tuning評価

In [1]:
# ライブラリのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install 'transformers==4.51.3'
!pip install 'accelerate==0.23.0'
!pip install 'peft==0.15.2'
!pip install 'trl==0.19.0'
!pip install 'bitsandbytes==0.45.5'
!pip install 'numpy==2.1.2'
!pip install 'tqdm==4.67.1'
!pip install 'scikit-learn==1.3.2'
!pip install 'pandas==2.2.3'

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.8.1
    Uninstalling accelerate-1.8.1:
      Successfully uninstalled accelerate-1.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.19.0 requires accelerate>=1.4.0, but you have accelerate 0.23.0 which is incompatible.
  Using cached accelerate-1.8.1-py3-none-any.whl (365 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.23.0
    Uninstalling accelerate-0.23.0:
      Successfully uninstalled accelerate-0.23.0
  Using cached numpy-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successf

In [2]:
# Jsonで変数を定義
valiables = {
 "instruct-tuning-checkpoint": "./output/llm-jp-3-1-1.8b-instruct4-8bit-lora/checkpoint-27000"
}

In [3]:
# 警告抑制
import warnings
warnings.simplefilter("ignore")

In [4]:
# モデルを設定
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# 量子化設定
# bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# トークナイザ, モデルの読み込み
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-3.1-1.8b-instruct4", use_fast=True)
# model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-3.1-1.8b-instruct4", quantization_config=bnb_config, device_map="cuda:0")
model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-3.1-1.8b-instruct4", device_map="cuda:0")
print(model)

# Checkpointの読み込み
peft_model = PeftModel.from_pretrained(model, valiables["instruct-tuning-checkpoint"])

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(99584, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=7168, bias=False)
          (up_proj): Linear(in_features=2048, out_features=7168, bias=False)
          (down_proj): Linear(in_features=7168, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb)

In [5]:
# 評価モードに設定
peft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(99584, 2048)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [7]:
# データセットの準備
import pandas as pd
from datasets import load_dataset, Dataset

# JCommonsenseQAをダウンロードし、データフレームに変換する
dataset_qa = load_dataset("shunk031/JGLUE", name="JCommonsenseQA")
train_df_qa = dataset_qa["train"].to_pandas()
test_df_qa = dataset_qa["validation"].to_pandas()

# JCommonsenseQAデータセットの整備
def preprocess_qa_df(df):
  qa_instruction_text = "質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。"
  df["instruction"] = qa_instruction_text
  df["input"] = df.apply(lambda x: f"質問:\n{x['question']}\n選択肢:\n0.{x['choice0']} 1.{x['choice1']} 2.{x['choice2']} 3.{x['choice3']} 4.{x['choice4']}", axis=1)
  df["output"] = df["label"].astype(str)
  df = df[["instruction", "input", "output"]]
  return df

train_df_qa = preprocess_qa_df(train_df_qa)
print(f"train_df_qa: {train_df_qa.head()}")
test_df_qa = preprocess_qa_df(test_df_qa)
print(f"test_df_qa: {test_df_qa.head()}")

# datasetクラスに変換する
dataset_train = Dataset.from_pandas(train_df_qa)
dataset_train = dataset_train.train_test_split(train_size=0.9, seed=1024)
dataset_test = Dataset.from_pandas(test_df_qa)

# 推論用にデータセット前処理
def add_text(example):
    example["text"] = f"### 指示:\n{example['instruction']}\n\n### 入力:\n{example['input']}\n\n### 回答:\n"
    return example

dataset_test = dataset_test.map(add_text)
print(f"dataset_test: {dataset_test}")

train_df_qa:                                          instruction  \
0  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
1  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
2  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
3  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
4  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   

                                               input output  
0  質問:\n主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？\n選択肢:...      2  
1  質問:\n未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人...      1  
2  質問:\n数字の１を表すときに使う体は？\n選択肢:\n0.胸 1.肉球 2.背中 3.人差...      3  
3  質問:\n火を起こすとあらわれるもくもくするものは？\n選択肢:\n0.歯の変色 1.ガス ...      4  
4  質問:\n大型商業施設といえば？\n選択肢:\n0.アパート 1.レジデンス 2.ドレッサー...      3  
test_df_qa:                                          instruction  \
0  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
1  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
2  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...   
3  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢

Map: 100%|██████████| 1119/1119 [00:00<00:00, 42209.73 examples/s]

dataset_test: Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 1119
})


In [11]:
# 推論実行
def generate(prompt, return_full_text=False, max_token=512):
    input_ids = tokenizer.encode(
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device=peft_model.device)
    output_ids = peft_model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.pad_token_id,
        max_length=max_token,
        temperature=0.1,
        num_return_sequences=1
    )
    if return_full_text:
        return tokenizer.decode(output_ids.tolist()[0])
    output = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1):])
    return output

prompt = """
### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
"""
print(generate(prompt, True))


### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
富士山</s>


In [8]:
# 精度の評価
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def predict(prompt):
    answer = generate(prompt, False)
    try:
      return int(answer[0])
    except ValueError:
      return -1

y_preds = []
for i in tqdm(range(len(dataset_test))):
    y_preds.append(predict(dataset_test["text"][i]))

y_true = dataset_test["output"]
y_true = [int(i) for i in y_true]

y_preds = np.array(y_preds)
y_true = np.array(y_true)

print(f"accuracy : {accuracy_score(y_true, y_preds):.3f}")

100%|██████████| 1119/1119 [00:56<00:00, 19.81it/s]

accuracy : 0.936


In [9]:
# 応用：回答の長さの制御(回答の長さwお制御する指示を与えて文章を生成)
prompt = """### 指示:\n 以下の質問に答えてください。\n ### 入力:\n アマゾン川はどこにありますか？\n ### 回答:\n"""
print(generate(prompt, True, 256))

### 指示:
 以下の質問に答えてください。
 ### 入力:
 アマゾン川はどこにありますか？
 ### 回答:
南アメリカ大陸の西部に位置する</s>


In [10]:
prompt = """### 指示:\n以下の質問に20字以内で答えてください。\n ### 入力:\n アマゾン川はどこにありますか？\n ### 回答:\n"""
print(generate(prompt, True, 256))

### 指示:
以下の質問に20字以内で答えてください。
 ### 入力:
 アマゾン川はどこにありますか？
 ### 回答:
南アメリカ</s>


In [23]:
# 応用：回答の形式の制御(回答の形式を指示するプロンプトを与える)
# ※InstructionTuningに含まれていない文言は再現不可
prompt = """### 指示:\n以下の質問に３つ答えてください。\n ### 入力:\n ブラジルの川は？\n ### 回答:\n"""
print(generate(prompt, True, 256))

### 指示:
以下の質問に３つ答えてください。
 ### 入力:
 ブラジルの川は？
 ### 回答:
アマゾン川</s>


In [12]:
# 応用：学習したタスクと同じタスクを解く
# ※学習データに入っているタスクと同じ指示を与えて文章生成させる
prompt = """### 指示:\n入力されたワードを説明してください。\n ### 入力:\n プログラミング\n ### 回答:\n"""
print(generate(prompt, True, 256))

### 指示:
入力されたワードを説明してください。
 ### 入力:
 プログラミング
 ### 回答:
プログラミング（）とは、コンピュータに命令を与えるための言語を用いて、コンピュータに処理をさせる行為である。</s>


In [13]:
prompt = """### 指示:\n入力されたワードを説明してください。\n ### 入力:\n 大規模言語モデル\n ### 回答:\n"""
print(generate(prompt, True, 512))

### 指示:
入力されたワードを説明してください。
 ### 入力:
 大規模言語モデル
 ### 回答:
大規模言語モデル（）は、大規模なコーパス（大量のテキストデータ）を学習したニューラルネットワークモデルである。</s>
